<h1>3장 대규모 언어 모델 자세히 살펴 보기</h1>
<i>생성 LLM을 위한 트랜스포머 아키텍처 탐험하기</i>

<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter03.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 3장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

In [1]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [2]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

# LLM 로드하기

| 항목                     | 설명                                     |
| ---------------------- | -------------------------------------- |
| `AutoTokenizer`        | 문장 ↔ 토큰 변환 담당                          |
| `AutoModelForCausalLM` | 텍스트 생성 모델(Phi-3) 로드                    |
| GPU 사용                 | `device_map="cuda"`                    |
| dtype 자동 설정            | `torch_dtype="auto"`                   |
| pipeline 목적            | 모델 + 토크나이저를 손쉽게 사용하도록 묶어줌              |
| 생성 옵션                  | `max_new_tokens=50`, `do_sample=False` |


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# -----------------------------------------------------------
# 1. 토크나이저 로드
#    - "microsoft/Phi-3-mini-4k-instruct" 모델에서 사용하는
#      토크나이저를 다운로드하고 메모리에 불러옵니다.
#    - 토크나이저는 문장을 '토큰(token)'이라는 숫자 목록으로 바꾸고,
#      다시 숫자를 문장으로 복원하는 역할을 합니다.
# -----------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# -----------------------------------------------------------
# 2. 사전학습 언어 모델 로드
#    - AutoModelForCausalLM : “문장을 생성하는 모델”을 불러올 때 사용
#    - device_map="cuda"     : GPU를 사용해 모델을 실행하여 빠르게 처리
#    - torch_dtype="auto"    : 모델이 자동으로 최적 데이터 타입 선택
#    - trust_remote_code=True:
#         일부 모델은 자체 커스텀 코드가 필요하므로,
#         Hugging Face Hub에 있는 모델 코드를 신뢰하고 실행하도록 설정
# -----------------------------------------------------------
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

# -----------------------------------------------------------
# 3. 텍스트 생성 파이프라인 생성
#    - pipeline("text-generation")
#         → 모델과 토크나이저를 묶어줘서
#           “문장을 이어서 생성하는 기능”을 쉽게 사용할 수 있게 해줍니다.
#
#    주요 옵션 설명:
#    - return_full_text=False :
#         입력(prompt) 문장을 출력에서 제외하고
#         새로 생성된 부분만 반환
#
#    - max_new_tokens=50 :
#         새로 만들어낼 문장의 최대 길이(토큰 수)
#
#    - do_sample=False :
#         랜덤 선택 없이 가장 가능성이 높은 단어만 고르는 방식
#         → 안정적, 반복 가능한 결과 생성
# -----------------------------------------------------------
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added mal

# 훈련된 트랜스포머 LLM의 입력과 출력


In [4]:
# -----------------------------------------------------------
# 1. 모델에게 보낼 요청(prompt) 작성
#    - 사용자가 모델에게 “무엇을 생성해 달라”고 지시하는 문장입니다.
#    - 여기서는 “정원에서 일어난 사고에 대해 사라에게 사과 이메일 작성”
#      요청을 모델에게 보냅니다.
# -----------------------------------------------------------
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

# -----------------------------------------------------------
# 2. 생성 파이프라인(generator)을 사용해 텍스트 생성
#    - generator(prompt) : 프롬프트를 넣으면 모델이 문장을 이어서 생성해 줍니다.
#    - output은 여러 개의 결과를 담은 리스트 형태로 반환됩니다.
# -----------------------------------------------------------
output = generator(prompt)

# -----------------------------------------------------------
# 3. 생성된 문장 출력
#    - output[0] : 첫 번째 생성 결과
#    - ['generated_text'] : 모델이 새로 만들어낸 텍스트 부분
# -----------------------------------------------------------
print(output[0]['generated_text'])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Mention the steps you're taking to prevent it in the future.

Dear Sarah,

I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in


| 항목    | 설명                                            |
| ----- | --------------------------------------------- |
| 출력 내용 | Phi-3 모델의 전체 구조(모듈, 블록, 레이어)                  |
| 포함 요소 | 임베딩, Transformer Layer, LayerNorm, lm_head    |
| 목적    | 모델이 어떤 네트워크로 구성되어 있는지 확인                      |
| 유용성   | 디버깅, 모델 이해, 구조 비교 등에 활용                       |
| 특징    | GPT 계열처럼 **Decoder-only Transformer** 구조로 표시됨 |


In [5]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

# 확률 분포로부터 하나의 토큰 선택하기(샘플링/디코딩)

| 항목                | 설명                                                         |
| ----------------- | ---------------------------------------------------------- |
| `input_ids`       | 프롬프트("The capital of France is")를 숫자 토큰으로 변환한 값            |
| `.to("cuda")`     | 입력을 GPU에서 처리하도록 이동                                         |
| `model.model()`   | Transformer 본체(Attention + FFN)만 실행하여 **hidden states** 생성 |
| `model_output[0]` | 각 토큰의 은닉 상태(hidden vector)                                 |
| `model.lm_head()` | hidden state를 vocabulary 크기의 **logits(단어 점수)** 로 변환        |
| 역할 분리             | `model.model` = 변환기 / `lm_head` = 단어 예측기                   |
| 전체 목적             | “다음 토큰이 될 단어 분포”를 내부 단계별로 직접 확인                            |


In [6]:
prompt = "The capital of France is"

# ---------------------------------------------------------
# 1. 입력 문장을 토큰화
#    - tokenizer()는 문장을 숫자 토큰(token)으로 변환합니다.
#    - return_tensors="pt" : PyTorch 텐서 형태로 반환
# ---------------------------------------------------------
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# ---------------------------------------------------------
# 2. 모델이 GPU를 사용하도록 입력을 GPU로 이동
# ---------------------------------------------------------
input_ids = input_ids.to("cuda")

# ---------------------------------------------------------
# 3. Transformer 본체(model.model)만 실행
#    - model.model :
#         Phi-3의 "Transformer 블록" 부분
#         (임베딩, Attention, FFN 등)
#
#    - 이 단계에서는 단순히 “다음 토큰 예측 전에 필요한 변환”
#      (즉, 은닉 상태 hidden states)만 계산합니다.
# ---------------------------------------------------------
model_output = model.model(input_ids)

# ---------------------------------------------------------
# 4. lm_head을 통해 실제 단어 예측 점수(logits) 계산
#    - model.lm_head :
#         Transformer가 계산한 hidden states를
#         실제 "단어 확률 벡터"로 변환하는 마지막 층
#
#    - lm_head_output :
#         각 토큰마다 vocabulary 전체에 대한 점수(logits)
# ---------------------------------------------------------
lm_head_output = model.lm_head(model_output[0])

| 코드                           | 의미                           |
| ---------------------------- | ---------------------------- |
| `lm_head_output[0, -1]`      | 마지막 토큰 위치의 단어 점수(logits)     |
| `.argmax(-1)`                | 가장 높은 점수를 가진 단어의 token ID 선택 |
| `tokenizer.decode(token_id)` | 숫자 ID → 실제 단어로 복원            |
| 전체 목적                        | 모델 내부에서 **다음 단어 직접 예측**하기    |
| 예시                           | `"Paris"`                    |


In [7]:
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

'Paris'

- 1 → batch size (입력 문장 수)

- sequence_length → 프롬프트가 토큰화된 토큰 개수
  - 예: "The capital of France is" → 5~9 tokens 정도

- hidden_size → Phi-3의 은닉 벡터 차원
  - (Phi-3 mini 모델은 약 3072차원 사용)

In [ ]:
model_output[0].shape

torch.Size([1, 5, 3072])

- 프롬프트의 각 토큰에 대해 다음 토큰을 무엇으로 예측할지 점수를 담은 데이터

- 즉,
입력된 문장의 각 위치에서 ‘다음에 올 단어 후보’ 전체에 대한 점수표

In [ ]:
lm_head_output.shape

torch.Size([1, 5, 32064])

# 키와 값을 캐싱하여 생성 속도 높이기


### LLM에서의 캐시(KV Cache)란?

- LLM이 문장을 생성할 때, Transformer의 Self-Attention은
“지금 생성하는 토큰이 앞의 모든 토큰을 참조”해야 한다.

- 즉,
- 1번째 토큰 생성 → 입력 전체 사용
- 2번째 토큰 생성 → 1번째 결과까지 포함
- 3번째 토큰 생성 → 1, 2번째 결과 포함
- …
- 100번째 토큰 생성 → 앞의 99개 토큰과 attention 계산

- 이렇게 되면 매 토큰마다 모든 과거 토큰과의 계산을 반복해야 하므로
속도가 크게 느려진다.

### 그 문제를 해결하는 기술이 바로 KV Cache (Key/Value Cache)

- Transformer의 Self-Attention에서 사용되는 요소:

  - Query (Q)

  - Key (K)

  - Value (V)

- 문장을 생성할 때 K와 V는 이전 토큰에서 이미 계산한 값들이다.

- 그래서, “매번 다시 계산하지 말고, 이전 단계에서 계산한 K/V 값을 저장해두자!”

- 이것이 바로 KV 캐싱이다.

| 설정                | 특징                           | 속도       |
| ----------------- | ---------------------------- | -------- |
| `use_cache=True`  | 캐시 활용 → 과거 계산 재사용            | 빠름 ⚡     |
| `use_cache=False` | 매 토큰마다 모든 attention 계산 다시 수행 | 매우 느림 🐢 |


In [8]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

# ---------------------------------------------------------
# 1. 프롬프트(입력 문장)를 토큰화
#    - tokenizer()는 문장을 숫자 토큰 배열로 변환합니다.
#    - return_tensors='pt' : PyTorch 텐서 형태로 변환
# ---------------------------------------------------------
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# ---------------------------------------------------------
# 2. 입력 토큰을 GPU로 이동
#    - 모델이 GPU에서 실행되므로 입력도 GPU로 보내야 합니다.
# ---------------------------------------------------------
input_ids = input_ids.to("cuda")

| 항목                   | 설명                           |
| -------------------- | ---------------------------- |
| `%%timeit -n 1`      | 셀 실행 시간을 한 번 측정              |
| `model.generate()`   | Phi-3 모델이 텍스트를 생성하는 함수       |
| `input_ids`          | 프롬프트를 토큰으로 변환한 값             |
| `max_new_tokens=100` | 새로 생성할 문장의 최대 길이             |
| `use_cache=True`     | Transformer 캐시 사용 → 생성 속도 증가 |
| 결과                   | 텍스트 생성에 걸리는 시간 + 생성된 토큰 결과   |


In [9]:
%%timeit -n 1
# ---------------------------------------------------------
# 1. %%timeit -n 1
#    - 이 셀을 실행하는 데 걸리는 시간을 측정하는 Jupyter 매직 명령어입니다.
#    - "-n 1"은 "한 번만 실행해서 시간을 재라"는 뜻입니다.
#    - 즉, 이 generate() 함수가 텍스트를 생성하는 데 얼마나 걸리는지
#      성능 테스트를 하는 코드입니다.
# ---------------------------------------------------------

# ---------------------------------------------------------
# 2. 텍스트 생성
#    - model.generate()는 LLM이 프롬프트(input_ids)에 이어서
#      새로운 텍스트를 생성하는 함수입니다.
#
#    주요 옵션 설명:
#    - max_new_tokens=100 :
#         새로 생성될 텍스트의 최대 길이(토큰 수)
#
#    - use_cache=True  :
#         Transformer의 캐시 기능을 사용해 더 빠르게 생성
#         (이전 생성 과정의 key/value를 재사용)
# ---------------------------------------------------------
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=True
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


5.11 s ± 289 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


| 항목                   | 설명                                        |
| -------------------- | ----------------------------------------- |
| `%%timeit -n 1`      | 셀 실행 시간을 한 번 측정                           |
| `max_new_tokens=100` | 100개의 새 토큰 생성                             |
| `use_cache=False`    | 캐시 비활성화 → 매 단계 전체 계산 반복 → **속도 매우 느림**    |
| 캐시란?                 | 이전 생성 단계의 키/값(attention 계산 결과)을 저장해 두는 기능 |
| 결과                   | LLM 생성 속도 실험(캐시 ON/OFF 비교 목적)             |


In [10]:
%%timeit -n 1
# ---------------------------------------------------------
# 1. %%timeit -n 1
#    - 이 Jupyter 매직 명령어는 셀 전체 실행 시간을 측정합니다.
#    - "-n 1"은 "1번만 실행해서 시간 측정"을 의미합니다.
#    - 즉, 텍스트 생성 속도를 테스트하기 위한 코드입니다.
# ---------------------------------------------------------

# ---------------------------------------------------------
# 2. 텍스트 생성
#    - model.generate()는 LLM이 프롬프트(input_ids)에 이어서
#      텍스트를 생성하는 함수입니다.
#
#    - 여기서 중요한 옵션:
#
#      use_cache=False
#        → Transformer의 “KV 캐시(Key/Value cache)” 기능을 끕니다.
#        → 캐시를 쓰지 않으면, 매 토큰 생성 시
#           이전 모든 token에 대한 연산을 다시 수행해야 하므로
#           훨씬 느려집니다.
#
#      max_new_tokens=100
#        → 새로 생성할 텍스트의 최대 길이
# ---------------------------------------------------------
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=False
)

32.3 s ± 306 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
